In [3]:
# import dependencies
import pandas as pd
import os
from sqlalchemy import create_engine


## Vehicle Registration Data

In [4]:
# read in vehicle registration CSV
reg_csv_file = "Vehicle__Snowmobile__and_Boat_Registrations.csv"
vehicle_reg_df = pd.read_csv(reg_csv_file)


In [5]:
# explore the dataframe: review list of columns
vehicle_reg_df.columns


Index(['Record Type', 'VIN', 'Registration Class', 'City', 'State', 'Zip',
       'County', 'Model Year', 'Make', 'Body Type', 'Fuel Type',
       'Unladen Weight', 'Maximum Gross Weight', 'Passengers',
       'Reg Valid Date', 'Reg Expiration Date', 'Color', 'Scofflaw Indicator',
       'Suspension Indicator', 'Revocation Indicator'],
      dtype='object')

In [6]:
# explore the dataframe: sample the top of the df
vehicle_reg_df.head()


,Record Type,VIN,Registration Class,City,State,Zip,County,Model Year,Make,Body Type,Fuel Type,Unladen Weight,Maximum Gross Weight,Passengers,Reg Valid Date,Reg Expiration Date,Color,Scofflaw Indicator,Suspension Indicator,Revocation Indicator
0,VEH,5NPD84LF0JH355353,PAS,LINDENHURST,NY,11757,SUFFOLK,2018.0,HYUND,4DSD,GAS,2976.0,NaN,NaN,11/19/2018,11/18/2020,BK,N,N,N
1,VEH,JF1GJAA66EH010986,PAS,SLEEPY HOLLOW,NY,10591,WESTCHESTER,2014.0,SUBAR,4DSD,GAS,3010.0,NaN,NaN,10/03/2017,10/02/2019,WH,N,N,N
2,VEH,5NPE24AF5FH208042,PAS,FAIRPORT,NY,14450,MONROE,2015.0,HYUND,4DSD,GAS,3252.0,NaN,NaN,11/02/2018,10/09/2020,GY,N,N,N
3,VEH,JF2GPAPC3F8320456,ORG,WESTBURY,NY,11590,NASSAU,2015.0,SUBAR,SUBN,GAS,3241.0,NaN,NaN,01/11/2019,09/06/2020,WH,N,N,N
4,VEH,5NPE24AF5FH207974,PAS,MECHANICVILLE,NY,12118,SARATOGA,2015.0,HYUND,4DSD,GAS,3252.0,NaN,NaN,03/05/2019,06/21/2020,GY,N,N,N


In [7]:
# explore the dataframe: determine how many Record Types exist
vehicle_reg_df.groupby(['Record Type']).count()


,VIN,Registration Class,City,State,Zip,County,Model Year,Make,Body Type,Fuel Type,Unladen Weight,Maximum Gross Weight,Passengers,Reg Valid Date,Reg Expiration Date,Color,Scofflaw Indicator,Suspension Indicator,Revocation Indicator
Record Type,,,,,,,,,,,,,,,,,,,
BOAT,359921,359921,359921,359921,359921,359921,359893,359921,359921,359921,0,0,0,359921,359921,359921,359921,359921,359921
SNOW,105451,105451,105451,105451,105451,105451,105451,105451,105451,105451,0,0,0,105451,105451,105451,105451,105451,105451
TRL,760594,760594,760594,760594,760594,760594,760594,760594,760594,760594,109183,651410,0,760594,760594,760594,760594,760594,760594
VEH,10713500,10713500,10713500,10713500,10713500,10713500,10713500,10713500,10713500,10713500,9610478,768654,334226,10713499,10524941,10713500,10713500,10713500,10713500


In [8]:
# retain only those rows with a record type VEH (we only want vehicles, vs boats etc)
# note that record type has 4 characters, so we need to leave a space after VEH
veh_reg_df = vehicle_reg_df[vehicle_reg_df["Record Type"]== "VEH "]
veh_reg_df.tail()


,Record Type,VIN,Registration Class,City,State,Zip,County,Model Year,Make,Body Type,Fuel Type,Unladen Weight,Maximum Gross Weight,Passengers,Reg Valid Date,Reg Expiration Date,Color,Scofflaw Indicator,Suspension Indicator,Revocation Indicator
10713495,VEH,JF2GPBBC0GH241471,PAS,SARATOGA SPRGS,NY,12866,SARATOGA,2016.0,SUBAR,SUBN,GAS,3451.0,NaN,NaN,05/09/2019,06/18/2021,BK,N,N,N
10713496,VEH,JF2GPBBC0GH226100,PAS,STATEN ISLAND,NY,10302,RICHMOND,2016.0,SUBAR,SUBN,GAS,3451.0,NaN,NaN,01/14/2019,01/30/2021,WH,N,N,N
10713497,VEH,JF2GPBBC0GH224315,PAS,ST LOUIS,MO,63101,WESTCHESTER,2016.0,SUBAR,SUBN,GAS,3451.0,NaN,NaN,02/13/2018,03/31/2020,WH,N,N,N
10713498,VEH,JF2GPBBC0GH203612,PAS,BROOKLYN,NY,11218,KINGS,2016.0,SUBAR,SUBN,GAS,3451.0,NaN,NaN,10/22/2017,12/03/2019,GR,N,N,N
10713499,VEH,JF2GPBBC0GH203576,PSD,MARCY,NY,13403,ONEIDA,2016.0,SUBAR,SUBN,GAS,3451.0,NaN,NaN,11/18/2015,NaN,WH,N,N,N


In [9]:
# drop columns not relevant to our analysis
registration_df = veh_reg_df.drop(columns=['Record Type',
                                           'Fuel Type', 
                                           'Maximum Gross Weight', 
                                           'Passengers',
                                           'Scofflaw Indicator'])


In [10]:
# we want to clean up "Body Type" for joining with other data
# let's start by reviewing how many body types exist
# decode the body types here: https://data.ny.gov/api/views/w4pv-hbkt/files/AUsdC2Y0iEymGyebFASIjDxZ7irrm1-_yS-o9qFzWTQ?download=true&filename=NYSDMV_VehicleSnowmobileAndBoat_Registration_Data%20Dictionary.pdf
registration_df["Body Type"].value_counts()


SUBN    4966416
4DSD    3589210
PICK     457423
2DSD     453108
MCY      325935
VAN      231510
CONV     178701
ATV      115525
DUMP      82336
BUS       60472
UTIL      41219
TRAC      36178
H/WH      25720
DELV      24813
FLAT      17463
TRLR      14131
TAXI      11075
P/SH      10104
STAK      10065
TANK       8539
TOW        6711
MOPD       6635
REFG       5381
SEDN       5139
RBM        4780
LTRL       4302
RD/S       2487
CMIX       2448
TR/C       2283
SEMI       1799
AMBU       1318
LSV        1225
S/SP        988
FIRE        981
LIM         879
T/CR        858
HRSE        840
SWT         775
SN/P        725
EMVR        566
W/DR        466
DCOM        370
CUST        341
????        270
W/SR        251
H/TR        218
R/RD        216
MCC          81
FPM          81
LSVT         41
H/IN         28
BOAT         26
TR/E         20
RPLC         12
LOCO         11
TRAV          3
POLE          2
Name: Body Type, dtype: int64

In [11]:
# we notice that there are still some boats and snowplows, so let's eliminate those
registration_df = registration_df[(registration_df['Body Type'] != 'TRAV') & 
                                 (registration_df['Body Type'] != 'BOAT')]


In [12]:
# let's aggregate some of the body types together and convert to readable names
# remember, values have 4 characters, so:
# let we need to add a space after any 3 character value, and let's rename so none are left
replace_values = {'4DSD' : 'SEDAN', '2DSD' : 'SEDAN', 'SEDN' : 'SEDAN',
                  'PICK' : 'PASS_TRUCK', 'VAN ' : 'PASS_TRUCK', 'LTRL' : 'PASS_TRUCK', 'SWT ' : 'PASS_TRUCK',
                  '????' : 'OTHER', 'RPLC' :'OTHER', 'CUST' : 'OTHER',
                  'AMBU' : 'EMERGENCY', 'FIRE' : 'EMERGENCY', 'DCOM' : 'EMERGENCY',
                  'CMIX' : 'MACHINE', 'P/SH' : 'MACHINE', 'S/SP' : 'MACHINE', 'SN/P' : 'MACHINE', 'EMVR' : 'MACHINE', 'W/DR' : 'MACHINE', 'W/SR': 'MACHINE', 'MCC ': 'MACHINE', 'FPM ' : 'MACHINE', 'LOCO' : 'MACHINE', 'RBM ' : 'MACHINE', 'RD/S' : 'MACHINE',
                  'DELV' : 'COMM_TRUCK', 'DUMP' : 'COMM_TRUCK', 'UTIL' : 'COMM_TRUCK', 'TRAC' : 'COMM_TRUCK', 'FLAT' : 'COMM_TRUCK', 'TRLR' : 'COMM_TRUCK', 'STAK' : 'COMM_TRUCK', 'TANK' : 'COMM_TRUCK', 'TOW ' : 'COMM_TRUCK', 'REFG' : 'COMM_TRUCK', 'TR/C' : 'COMM_TRUCK', 'SEMI' : 'COMM_TRUCK', 'T/CR' : 'COMM_TRUCK', 'H/TR' : 'COMM_TRUCK', 'R/RD' : 'COMM_TRUCK', 'TR/E' : 'COMM_TRUCK', 'POLE' : 'COMM_TRUCK',
                  'HRSE' : 'HEARSE', 'H/IN' : 'HEARSE',
                  'H/WH' : 'HOUSE_ON_WHEELS',
                  'LIM ' : 'TAXI',
                  'LSV ' : 'LOW_SPEED_VEH', 'LSVT': 'LOW_SPEED_VEH',
                  'MCY ' : 'CYCLES', 'MOPD' : 'CYCLES',
                  'SUBN' : 'SUBURBAN',
                  'ATV ' : 'ATV',
                  'BUS ' : 'BUS'
                 }
new_registration_df = registration_df.replace({'Body Type': replace_values}) 


In [13]:
# review updated list of body types
new_registration_df['Body Type'].value_counts()


SUBURBAN           4966416
SEDAN              4047457
PASS_TRUCK          694010
CYCLES              332570
COMM_TRUCK          252232
CONV                178701
ATV                 115525
BUS                  60472
HOUSE_ON_WHEELS      25720
MACHINE              22988
TAXI                 11954
EMERGENCY             2669
LOW_SPEED_VEH         1266
HEARSE                 868
OTHER                  623
Name: Body Type, dtype: int64

In [14]:
# set vehicle registration number (VIN) as index 
new_registration_df.set_index('VIN', inplace=True)
new_registration_df.head()


,Registration Class,City,State,Zip,County,Model Year,Make,Body Type,Unladen Weight,Reg Valid Date,Reg Expiration Date,Color,Suspension Indicator,Revocation Indicator
VIN,,,,,,,,,,,,,,
5NPD84LF0JH355353,PAS,LINDENHURST,NY,11757,SUFFOLK,2018.0,HYUND,SEDAN,2976.0,11/19/2018,11/18/2020,BK,N,N
JF1GJAA66EH010986,PAS,SLEEPY HOLLOW,NY,10591,WESTCHESTER,2014.0,SUBAR,SEDAN,3010.0,10/03/2017,10/02/2019,WH,N,N
5NPE24AF5FH208042,PAS,FAIRPORT,NY,14450,MONROE,2015.0,HYUND,SEDAN,3252.0,11/02/2018,10/09/2020,GY,N,N
JF2GPAPC3F8320456,ORG,WESTBURY,NY,11590,NASSAU,2015.0,SUBAR,SUBURBAN,3241.0,01/11/2019,09/06/2020,WH,N,N
5NPE24AF5FH207974,PAS,MECHANICVILLE,NY,12118,SARATOGA,2015.0,HYUND,SEDAN,3252.0,03/05/2019,06/21/2020,GY,N,N


In [15]:
# review column names  
new_registration_df.columns


Index(['Registration Class', 'City', 'State', 'Zip', 'County', 'Model Year',
       'Make', 'Body Type', 'Unladen Weight', 'Reg Valid Date',
       'Reg Expiration Date', 'Color', 'Suspension Indicator',
       'Revocation Indicator'],
      dtype='object')

In [164]:
# we attempted to load dataframe with .to_sql, but it is too large for this, 
# so we will output to csv instead and import 

# create database connection
# connection_string = "postgres:chumwater@localhost:5432/ny_vehicles_db"
# engine = create_engine(f'postgresql://{connection_string}')

# Confirm table exists
# engine.table_names()

# load dataframe into database
# new_registration_df.to_sql(name='ny_vehicle_registration', con=engine, if_exists='append', index=True)


In [17]:
# output dataframe to CSV

new_registration_df.to_csv("ny_registration.csv", index=True, header=True)


## Vehicle Collision Data

In [6]:
collision_csv_file = "NYPD_Motor_Vehicle_Collisions.csv"
vehicle_collision_df = pd.read_csv(collision_csv_file)


/Users/scottgregory/anaconda2/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
vehicle_collision_df.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,07/12/2019,0:00,BRONX,10472,40.831623,-73.86726,"(40.831623, -73.86726)",WESTCHESTER AVENUE,SAINT LAWRENCE AVENUE,NaN,...,NaN,NaN,NaN,NaN,4168556,Sedan,NaN,NaN,NaN,NaN
1,07/12/2019,0:00,BROOKLYN,11207,40.669792,-73.89240,"(40.669792, -73.8924)",SUTTER AVENUE,WYONA STREET,NaN,...,Unspecified,NaN,NaN,NaN,4169745,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN
2,07/12/2019,0:00,BROOKLYN,11234,40.618717,-73.93262,"(40.618717, -73.93262)",NaN,NaN,2098 FLATBUSH AVENUE,...,Unspecified,NaN,NaN,NaN,4169938,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
3,07/12/2019,0:00,NaN,NaN,40.829155,-73.93728,"(40.829155, -73.93728)",8 AVENUE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4169906,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
4,07/12/2019,0:00,NaN,NaN,40.718987,-73.94635,"(40.718987, -73.94635)",BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4168595,Sedan,Sedan,NaN,NaN,NaN
